In [32]:
#load the labels
import os
import pandas as pd
import numpy as np
import cv2
import matplotlib.pyplot as plt
import shutil


#time is in the form XXH:XXm:XXs, so we need to convert it to seconds
def convert_time(time):
    hours = int(time[0:2])
    mins = int(time[4:6])
    secs = int(time[8:10])
    return hours*3600 + mins*60 + secs

#Preprocess the table
phases = pd.read_excel('../data/phases.xlsx', engine='openpyxl')

#strip leading and trailing whitespace from the time_start and time_end columns
phases['time_start'] = phases['time_start'].str.strip()
phases['time_end'] = phases['time_end'].str.strip()

#check if the time_start and time_end columns are in the correct format
for i, row in phases.iterrows():
    if len(row['time_start']) != 11:
        print(i, row['vid_id'], row['time_start'])
    if len(row['time_end']) != 11:
        print(i, row['vid_id'], row['time_end'])


# %%
phases = phases.sort_values(by=['vid_id'], ascending=False)
non_kc_vid_4 = phases[phases['vid_id'] == 'vid_4']
non_kc_vid_4 = non_kc_vid_4[non_kc_vid_4['labeler'] != 'kc']
phases = phases.drop(non_kc_vid_4.index)
phases.reset_index(drop=True, inplace=True)
phases.loc[phases['phase'] == 'sac_reduction', 'phase'] = 'hiatal_dissec'

# %%

#if vid_id contains 'vid', then path starts with 'unc/raw_ids', elif vid_id contains 'yale', then path starts with 'yale/raw_ids'
for i, row in phases.iterrows():
    if 'vid' in row['vid_id']:
        phases.loc[i, 'path'] = '../data/unc/raw_ids/' + row['vid_id'] + '.mp4'
    elif 'yale' in row['vid_id']:
        phases.loc[i, 'path'] = '../data/yale/raw_ids/' + row['vid_id'] + '.mp4'
    elif 'rush' in row['vid_id']:
        phases.loc[i, 'path'] = '../data/rush/raw_ids/' + (row['vid_id'].replace('0', '')) + '.mp4'
    elif 'UNC' in row['vid_id']:
        phases.loc[i, 'path'] = '../data/unc/raw_ids/vid_' + (row['vid_id'].split('_')[-1]) + '.mp4'
    else:
        print(row['vid_id'])
phases.loc[phases['path'] == '../data/rush/raw_ids/rush_.mp4', 'path'] = '../data/rush/raw_ids/rush_0.mp4'
phases.loc[phases['path'] == '../data/unc/raw_ids/vid_30.mp4', 'path'] = '../data/unc/raw_ids/vid_30.mpg'
#for each video, check if vid_{}_v2.mp4 exists, if it does, then replace the path with that
for i, row in phases.iterrows():
    if os.path.exists('../data/unc/raw_ids/' + row['vid_id'] + '_v2.mp4'):
        phases.loc[i, 'path'] = '../data/unc/raw_ids/' + row['vid_id'] + '_v2.mp4'
    elif os.path.exists('../data/yale/raw_ids/' + row['vid_id'] + '_v2.mp4'):
        phases.loc[i, 'path'] = '../data/yale/raw_ids/' + row['vid_id'] + '_v2.mp4'
    elif os.path.exists('../data/yale/raw_ids/' + row['vid_id'] + '_robo.mp4'):
        phases.loc[i, 'path'] = '../data/yale/raw_ids/' + row['vid_id'] + '_robo.mp4'
    else:
        pass


# %%
phases = phases[['vid_id', 'path', 'phase', 'time_start', 'time_end']]

/nas/longleaf/rhel8/apps/anaconda/2021.11.ood/lib/python3.9/site-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


In [34]:
phases

,vid_id,path,phase,time_start,time_end
0,yale_9,../data/yale/raw_ids/yale_9.mp4,oob,00H:18m:09s,00H:18m:27s
1,yale_9,../data/yale/raw_ids/yale_9.mp4,wrap,00H:52m:44s,01H:04m:06s
2,yale_9,../data/yale/raw_ids/yale_9.mp4,hiatal_repair,00H:50m:12s,00H:51m:42s
3,yale_9,../data/yale/raw_ids/yale_9.mp4,hiatal_dissec,00H:49m:21s,00H:50m:11s
4,yale_9,../data/yale/raw_ids/yale_9.mp4,other,00H:47m:50s,00H:49m:20s
...,...,...,...,...,...
901,UNC_26,../data/unc/raw_ids/vid_26.mp4,other,01H:52m:52s,01H:55m:06s
902,UNC_26,../data/unc/raw_ids/vid_26.mp4,oob,01H:55m:07s,01H:55m:13s
903,UNC_26,../data/unc/raw_ids/vid_26.mp4,other,01H:55m:14s,01H:58m:31s
904,UNC_26,../data/unc/raw_ids/vid_26.mp4,oob,01H:58m:32s,01H:59m:09s


In [37]:
vid_list = ['vid_14', 'vid_21', 'rush_02', 'vid_4', 'vid_18','rush_0', 'vid_24', 'vid_7', 'vid_17', 'vid_12', 'vid_20', 'UNC_29', 'vid_22', 'vid_11', 'rush_01', 'vid_13', 'UNC_27', 'vid_0', 'vid_3', 'vid_1', 'vid_6', 'vid_15', 'UNC_26','UNC_28', 'vid_19', 'vid_5', 'vid_16', 'vid_23', 'UNC_30']
phases_use = phases.loc[phases['vid_id'].isin(vid_list)]

In [42]:
phases_use = phases_use.reset_index(drop=True)
phases_use

,vid_id,path,phase,time_start,time_end
0,vid_7,../data/unc/raw_ids/vid_7_v2.mp4,oob,00H:00m:00s,00H:02m:40s
1,vid_7,../data/unc/raw_ids/vid_7_v2.mp4,exposure,00H:02m:41s,00H:03m:51s
2,vid_7,../data/unc/raw_ids/vid_7_v2.mp4,sac_excision,00H:59m:45s,01H:01m:35s
3,vid_7,../data/unc/raw_ids/vid_7_v2.mp4,eso_mob,01H:06m:29s,01H:07m:24s
4,vid_7,../data/unc/raw_ids/vid_7_v2.mp4,sac_excision,00H:41m:44s,00H:42m:49s
...,...,...,...,...,...
678,UNC_26,../data/unc/raw_ids/vid_26.mp4,other,01H:52m:52s,01H:55m:06s
679,UNC_26,../data/unc/raw_ids/vid_26.mp4,oob,01H:55m:07s,01H:55m:13s
680,UNC_26,../data/unc/raw_ids/vid_26.mp4,other,01H:55m:14s,01H:58m:31s
681,UNC_26,../data/unc/raw_ids/vid_26.mp4,oob,01H:58m:32s,01H:59m:09s


In [44]:
def convert_time(time):
    hours = int(time[0:2])
    mins = int(time[4:6])
    secs = int(time[8:10])
    return hours*3600 + mins*60 + secs

phases_use['time_start_sec'] = phases_use.apply(lambda row: convert_time(row['time_start']), axis=1)
phases_use['time_end_sec'] = phases_use.apply(lambda row: convert_time(row['time_end']), axis=1)

In [45]:
phases_use

,vid_id,path,phase,time_start,time_end,time_start_sec,time_end_sec
0,vid_7,../data/unc/raw_ids/vid_7_v2.mp4,oob,00H:00m:00s,00H:02m:40s,0,160
1,vid_7,../data/unc/raw_ids/vid_7_v2.mp4,exposure,00H:02m:41s,00H:03m:51s,161,231
2,vid_7,../data/unc/raw_ids/vid_7_v2.mp4,sac_excision,00H:59m:45s,01H:01m:35s,3585,3695
3,vid_7,../data/unc/raw_ids/vid_7_v2.mp4,eso_mob,01H:06m:29s,01H:07m:24s,3989,4044
4,vid_7,../data/unc/raw_ids/vid_7_v2.mp4,sac_excision,00H:41m:44s,00H:42m:49s,2504,2569
...,...,...,...,...,...,...,...
678,UNC_26,../data/unc/raw_ids/vid_26.mp4,other,01H:52m:52s,01H:55m:06s,6772,6906
679,UNC_26,../data/unc/raw_ids/vid_26.mp4,oob,01H:55m:07s,01H:55m:13s,6907,6913
680,UNC_26,../data/unc/raw_ids/vid_26.mp4,other,01H:55m:14s,01H:58m:31s,6914,7111
681,UNC_26,../data/unc/raw_ids/vid_26.mp4,oob,01H:58m:32s,01H:59m:09s,7112,7149


In [46]:
phases_use = phases_use.sort_values(by=['vid_id', 'time_start_sec'], ascending=[True, True])

In [50]:
phases_use = phases_use.reset_index(drop=True)

In [51]:
phases_use

,vid_id,path,phase,time_start,time_end,time_start_sec,time_end_sec
0,UNC_26,../data/unc/raw_ids/vid_26.mp4,oob,00H:00m:00s,00H:03m:13s,0,193
1,UNC_26,../data/unc/raw_ids/vid_26.mp4,exposure,00H:03m:14s,00H:05m:20s,194,320
2,UNC_26,../data/unc/raw_ids/vid_26.mp4,oob,00H:05m:21s,00H:05m:44s,321,344
3,UNC_26,../data/unc/raw_ids/vid_26.mp4,exposure,00H:05m:45s,00H:11m:07s,345,667
4,UNC_26,../data/unc/raw_ids/vid_26.mp4,hiatal_dissec,00H:11m:08s,00H:20m:19s,668,1219
...,...,...,...,...,...,...,...
678,vid_7,../data/unc/raw_ids/vid_7_v2.mp4,wrap,01H:41m:37s,02H:03m:34s,6097,7414
679,vid_7,../data/unc/raw_ids/vid_7_v2.mp4,other,02H:03m:35s,02H:08m:01s,7415,7681
680,vid_7,../data/unc/raw_ids/vid_7_v2.mp4,oob,02H:08m:02s,02H:08m:11s,7682,7691
681,vid_7,../data/unc/raw_ids/vid_7_v2.mp4,other,02H:08m:12s,02H:14m:09s,7692,8049


In [52]:
phases_use['vid_id'].value_counts()

vid_7      40
vid_11     37
vid_12     37
vid_23     36
UNC_26     31
UNC_28     31
UNC_27     30
vid_14     29
vid_6      28
vid_21     28
vid_15     27
vid_19     27
vid_1      25
rush_0     23
vid_13     22
UNC_29     22
vid_18     22
vid_0      21
vid_17     20
vid_22     20
vid_16     19
vid_5      18
vid_24     17
vid_20     16
UNC_30     15
rush_01    14
rush_02    14
vid_4      11
vid_3       3
Name: vid_id, dtype: int64

In [55]:
phases_use[phases_use['vid_id']=='vid_3']

,vid_id,path,phase,time_start,time_end,time_start_sec,time_end_sec
583,vid_3,../data/unc/raw_ids/vid_3.mp4,exposure,00H:00m:00s,00H:02m:05s,0,125
584,vid_3,../data/unc/raw_ids/vid_3.mp4,hiatal_dissec,00H:02m:06s,00H:03m:00s,126,180
585,vid_3,../data/unc/raw_ids/vid_3.mp4,exposure,00H:03m:01s,00H:11m:59s,181,719


In [54]:
phases_use

,vid_id,path,phase,time_start,time_end,time_start_sec,time_end_sec
0,UNC_26,../data/unc/raw_ids/vid_26.mp4,oob,00H:00m:00s,00H:03m:13s,0,193
1,UNC_26,../data/unc/raw_ids/vid_26.mp4,exposure,00H:03m:14s,00H:05m:20s,194,320
2,UNC_26,../data/unc/raw_ids/vid_26.mp4,oob,00H:05m:21s,00H:05m:44s,321,344
3,UNC_26,../data/unc/raw_ids/vid_26.mp4,exposure,00H:05m:45s,00H:11m:07s,345,667
4,UNC_26,../data/unc/raw_ids/vid_26.mp4,hiatal_dissec,00H:11m:08s,00H:20m:19s,668,1219
...,...,...,...,...,...,...,...
678,vid_7,../data/unc/raw_ids/vid_7_v2.mp4,wrap,01H:41m:37s,02H:03m:34s,6097,7414
679,vid_7,../data/unc/raw_ids/vid_7_v2.mp4,other,02H:03m:35s,02H:08m:01s,7415,7681
680,vid_7,../data/unc/raw_ids/vid_7_v2.mp4,oob,02H:08m:02s,02H:08m:11s,7682,7691
681,vid_7,../data/unc/raw_ids/vid_7_v2.mp4,other,02H:08m:12s,02H:14m:09s,7692,8049


In [56]:
phases_use.to_excel('../data/phases_use.xlsx')

In [19]:
import numpy as np
import os

In [20]:
vid = np.load('../results/extracted_fe_slowfast_xception_070623/X_vid_13.npy')

In [21]:
new_vid = []
for clip in vid:
    cur_frame = clip[15]
    clip = np.append(clip, np.array([cur_frame]), axis=0)
    new_vid.append(clip)
new_vid = np.array(new_vid)

In [22]:
new_vid.shape

(510, 33, 2048)